<a href="https://colab.research.google.com/github/Purukan-Jeremy/Sentiment_Analysis_on_Game_Reviews/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SENTIMENT ANALYSIS PURUKAN JEREMY**

In [2]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import math
from collections import Counter
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.tree import export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import multilabel_confusion_matrix
from statistics import mean
import joblib


In [3]:
import time
start_time = time.time()


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

input_path = '/content/drive/MyDrive/Game Sentiment/data processed/'
output_path = '/content/drive/MyDrive/Game Sentiment/data sentiment/'

files = [
    'genshin_processed.csv',
    'hsr_processed.csv',
    'zenless_processed.csv',
    'wuwa_processed.csv',
    'blue_processed.csv'
]

# Combine all files
combined_df = pd.concat([pd.read_csv(input_path + f) for f in files], ignore_index=True)

combined_df = combined_df[combined_df['score'] != 3]

combined_df['class'] = combined_df['score'].apply(lambda x: 0 if x >= 4 else 1)

cols = ['class'] + [col for col in combined_df.columns if col != 'class']
combined_df = combined_df[cols]

combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into train/test sets
train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)

# Save to CSV
train_df.to_csv(output_path + 'train_all.csv', index=False, encoding='utf-8')
test_df.to_csv(output_path + 'test_all.csv', index=False, encoding='utf-8')


In [5]:
df_train = pd.read_csv(output_path + 'train_all.csv',header=0)
df_test = pd.read_csv(output_path + 'test_all.csv',header=0)

df_train = df_train.dropna(subset=['content'])
df_test = df_test.dropna(subset=['content'])

all_label = set(df_train['class'].tolist())

dst_train = Counter(df_train['class'].tolist())

df_train

,class,reviewId,content,score,at
0,1,485bceb8-d3ec-4f85-a86e-26e15452f2a7,fix game update game become lag always get for...,1,2024-06-29 11:26:58
1,0,e534f49a-22e2-4878-babe-b5a1be55f25c,far one wholesom experi joy play mobil especia...,5,2020-10-05 15:38:52
2,1,5211467f-59d7-4999-b424-1a10b743a335,anno domini skip buton make where are you make...,1,2025-01-01 01:10:13
3,1,f78da3b1-62f6-4938-91df-adee6e355aa2,story god lost hype god story incredibly bad s...,1,2025-01-29 13:46:26
4,0,0c92ce95-a313-4f72-8a15-23dd2152043b,star por optimization brand new phone laptop t...,4,2025-02-03 15:31:36
...,...,...,...,...,...
26890,1,7149cedf-98e1-4beb-9602-d20ddf2b2252,cant even log tried updating buton log scren e...,1,2023-11-05 03:08:36
26891,1,297bce7d-e864-406a-90f7-b30bdf6ad9b1,game might god distinguish art music would put...,1,2024-08-09 16:28:33
26892,0,8b4dcecf-3305-419d-a8d6-3c65d3556ab4,realy god game like lot like story except xian...,4,2024-09-28 12:42:54
26893,0,96506b92-c2bd-4c71-bcb8-97b4e1df37cc,realy like overal mechanic game using mobile p...,5,2024-05-31 16:16:50


In [6]:
#corpus
corpus_data = pd.concat([df_train, df_test], ignore_index=True)
corpus = corpus_data['content'].to_numpy()

#1-gram
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))
vectorizer.fit(corpus)

print("Vocabulary: ", vectorizer.vocabulary_)

Vocabulary:  {'fix': 7315, 'game': 7953, 'update': 20874, 'become': 1773, 'lag': 11007, 'always': 644, 'get': 8197, 'forced': 7502, 'minute': 12421, 'log': 11472, 'even': 6530, 'play': 14508, 'point': 14640, 'daily': 4512, 'mision': 12457, 'isue': 10455, 'take': 19214, 'mb': 11999, 'stil': 18536, 'fixed': 7322, 'anoying': 813, 'far': 7023, 'one': 13512, 'wholesom': 21588, 'experi': 6749, 'joy': 10660, 'mobil': 12527, 'especiali': 6466, 'arpg': 1092, 'art': 1099, 'stune': 18774, 'music': 12814, 'imers': 9675, 'best': 1901, 'al': 516, 'gameplay': 7979, 'fluid': 7426, 'except': 6636, 'command': 3588, 'offic': 13449, 'correct': 4085, 'compani': 3606, 'counter': 4119, 'offer': 13448, 'origin': 13676, 'poster': 14753, 'dungeon': 5828, 'world': 21776, 'truli': 20117, 'fel': 7136, 'open': 13546, 'wori': 21761, 'lost': 11543, 'progr': 15076, 'restart': 16336, 'due': 5801, 'conect': 3797, 'isu': 10453, 'beautifuli': 1754, 'deliv': 4799, 'complet': 3678, 'mani': 11838, 'fail': 6938, 'nowaday': 13

In [7]:
#test data
test_data = vectorizer.transform(df_test['content']).toarray()
feature_test = pd.DataFrame(data=test_data, columns=vectorizer.get_feature_names_out())

#train data
train_data = vectorizer.transform(df_train['content']).toarray()
feature_train = pd.DataFrame(data=train_data, columns=vectorizer.get_feature_names_out())

#features
print(feature_test)

       ab  aback  abadon  abandon  abandoned  abandoning  aber  abhor  \
0     0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
1     0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
2     0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
3     0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
4     0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
...   ...    ...     ...      ...        ...         ...   ...    ...   
6703  0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
6704  0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
6705  0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
6706  0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   
6707  0.0    0.0     0.0      0.0        0.0         0.0   0.0    0.0   

      abhorent  abi  ...  zoneles  zongli  zoning   zr   zu  zuck  zuschauen  \
0          0.0  0.0  ...      0.0     0.0  

In [8]:
class_train = df_train['class'];
feature_train['class'] = class_train

class_test = df_test['class'];
feature_test['class'] = class_test

feature_test = feature_test.dropna()
feature_train = feature_train.dropna()

cols = [col for col in feature_train.columns if col != 'class'] + ['class']
feature_train = feature_train[cols]

cols = [col for col in feature_test.columns if col != 'class'] + ['class']
feature_test = feature_test[cols]

feature_test

,ab,aback,abadon,abandon,abandoned,abandoning,aber,abhor,abhorent,abi,...,zongli,zoning,zr,zu,zuck,zuschauen,zwar,zxo,zzz,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
x_test = feature_test.iloc[:, :-1]
y_test = feature_test['class'].tolist()

x_train = feature_train.iloc[:, :-1]
y_train = feature_train['class'].tolist()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (26831, 22094)
26831 train samples
6692 test samples


In [10]:
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=2000)  # atau 2000, dsb
x_train = selector.fit_transform(x_train, y_train)
x_test = selector.transform(x_test)

In [11]:
from sklearn.utils import class_weight
import numpy as np
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)  # Karena TF-IDF hasilnya sparse
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

## **PERCEPTRON**

In [12]:
#training the algorithm

model_clf = Perceptron(max_iter=100, eta0=0.9, random_state=0)

from sklearn.calibration import CalibratedClassifierCV
model_clf = CalibratedClassifierCV(model_clf, method='isotonic')

model_clf.fit(x_train, y_train)

CalibratedClassifierCV(estimator=Perceptron(eta0=0.9, max_iter=100),
                       method='isotonic')

In [13]:
import pickle
filename = '/content/drive/MyDrive/Game Sentiment/data sentiment/perceptron_classifier.model'
pickle.dump(model_clf, open(filename + 'model_clf.sav', 'wb'))

## **RANDOM FOREST**

In [14]:
from sklearn.ensemble import RandomForestClassifier

model_clf = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=0)

model_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [16]:
import pickle
filename = '/content/drive/MyDrive/Game Sentiment/data sentiment/randomforest.model'
pickle.dump(model_clf, open(filename + 'model_clf.sav', 'wb'))

## **GRID**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Buat pipeline
pipeline = Pipeline([
    ('clf', RandomForestClassifier(random_state=0))
])

# Definisikan parameter grid untuk Random Forest
param_grid = {
    'clf__n_estimators': [100, 200, 300],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

model_clf = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Jalankan grid search di data training
model_clf.fit(x_train, y_train)

# Hasil terbaik
#print("Best Parameters:", grid_search.best_params_)
#print("Best Score (Accuracy):", grid_search.best_score_)

# Uji pada data test
#y_pred = model_clf.predict(x_test)
#print("\nClassification Report:\n", classification_report(y_test, y_pred))

Fitting 5 folds for each of 108 candidates, totalling 540 fits


## **SVM**

In [13]:
from sklearn.svm import SVC

model_clf = SVC(kernel='linear', C=1.0, random_state=0)

model_clf.fit(x_train, y_train)


SVC(kernel='linear', random_state=0)

In [20]:
import pickle
filename = '/content/drive/MyDrive/Game Sentiment/data sentiment/SVM.model'
pickle.dump(model_clf, open(filename + 'model_clf.sav', 'wb'))

## **KNN**

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

model_clf = KNeighborsClassifier(n_neighbors=5)

model_clf.fit(x_train, y_train)

KNeighborsClassifier()

In [22]:
import pickle
filename = '/content/drive/MyDrive/Game Sentiment/data sentiment/KNN.model'
pickle.dump(model_clf, open(filename + 'model_clf.sav', 'wb'))

## **NAIVE**

In [23]:
from sklearn.naive_bayes import MultinomialNB

model_clf = MultinomialNB()
model_clf.fit(x_train, y_train)

MultinomialNB()

In [24]:
import pickle
filename = '/content/drive/MyDrive/Game Sentiment/data sentiment/naive.model'
pickle.dump(model_clf, open(filename + 'model_clf.sav', 'wb'))

## **LOGISTIC**

In [25]:
from sklearn.linear_model import LogisticRegression

model_clf = LogisticRegression(max_iter=100, random_state=0)

model_clf.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [26]:
import pickle
filename = '/content/drive/MyDrive/Game Sentiment/data sentiment/logistic.model'
pickle.dump(model_clf, open(filename + 'model_clf.sav', 'wb'))

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the input shape
input_dim = x_train.shape[1]

# Build the SLNN model
model_clf = Sequential([
    Dense(1, input_dim=input_dim, activation='sigmoid')  # Single-layer NN
])


model_clf.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

y_train = np.array(y_train)

# Train the model
history = model_clf.fit(x_train, y_train, epochs=1, batch_size=64, validation_split=0.2, verbose=1)


336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5267 - loss: 0.6910 - val_accuracy: 0.5797 - val_loss: 0.6842


## **RUN MODEL**

In [14]:
y_pred = model_clf.predict(x_test)

print(y_pred)

[1. 1. 0. ... 1. 0. 1.]


In [15]:
lst_actual_class = y_test
lst_predicted_class = y_pred

#label ['POSITIF', 'NETRAL','NEGATIF'], [0, 1, 2]
lst_classes = [0, 1]

##Compute binary-class confusion matrix
tn, fp, fn, tp = confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes).ravel()

print("Unique actual labels:", set(lst_actual_class))
print("Unique predicted labels:", set(lst_predicted_class))

sensitivity = round(tp / (tp + fn), 3)
specificity = round(tn / (tn + fp), 3)
accuracy = round((tp + tn) / (tp + fp + tn + fn), 3)
balanced_accuracy = round((sensitivity + specificity) / 2, 3)
precision = round(tp / (tp + fp), 3)
fscore = round((2 * ((precision * sensitivity) / (precision + sensitivity))), 3)
mcc = round((((tp * tn) - (fp * fn)) /
             math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))), 3)

print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn))
print("Sensitivity: {0}".format(sensitivity))
print("Specificity: {0}".format(specificity))
print("Accuracy: {0}".format(accuracy))
print("Balanced Accuracy: {0}".format(balanced_accuracy))
print("Precision: {0}".format(precision))
print("F1-Score: {0}".format(fscore))
print("MCC: {0}\n".format(mcc))

Unique actual labels: {0.0, 1.0}
Unique predicted labels: {np.float64(0.0), np.float64(1.0)}
TP=1223, FP=1253, TN=2116, FN=2100
Sensitivity: 0.368
Specificity: 0.628
Accuracy: 0.499
Balanced Accuracy: 0.498
Precision: 0.494
F1-Score: 0.422
MCC: -0.004

